In [123]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup
from random import choices, seed, randrange
import time

In [113]:
df = pd.DataFrame(columns=["seed_title", "seed_url", "seed_creator", "rec_title", "rec_url", "rec_creator"])
df

,seed_title,seed_url,seed_creator,rec_title,rec_url,rec_creator


In [107]:
def get_title(span):
    return span['title']

def get_url(span):
    return 'https://www.youtube.com/' + span.find_parent('a', "yt-simple-endpoint")['href']

def get_creator(span):
    return span.find_parent('a', "yt-simple-endpoint").find('yt-formatted-string').get_text()

def get_seed_title(soup):
    return soup.find('h1', 'title').get_text()

def get_seed_creator(soup):
    return soup.find('ytd-channel-name').find('yt-formatted-string').get_text()

In [122]:
counter = 0
seed()
def get_vid_recs(seed_url):
    global counter
    driver.get(seed_url)
    time.sleep(randrange(7, 20))
    soup = BeautifulSoup(driver.page_source)
    seed_title = get_seed_title(soup)
    seed_creator = get_seed_creator(soup)
    rec_spans = soup.find_all('span', id='video-title')[:20]
    chosen = choices(rec_spans, k=5)
    for sp in chosen:
        info = {
            "seed_title": seed_title, 
            "seed_url": seed_url, 
            "seed_creator": seed_creator, 
            "rec_title": get_title(sp), 
            "rec_url": get_url(sp), 
            "rec_creator": get_creator(sp)
        }
        print(info)
        df.loc[counter] = info
        counter += 1
    rec_url = get_url(choices(chosen)[0])
    print("rec_url", rec_url)
    if counter < 100:
        get_vid_recs(rec_url)

In [120]:
driver = webdriver.Chrome()

In [132]:
get_vid_recs('https://www.youtube.com//watch?v=m2zZMg7SNWA')

{'seed_title': 'Political Correctness and Vacuous Wokeness: Douglas Murray debates Sylvana Simons', 'seed_url': 'https://www.youtube.com//watch?v=m2zZMg7SNWA', 'seed_creator': 'Sky News Australia', 'rec_title': 'Lectures: Exploring the Psychology of Creativity', 'rec_url': 'https://www.youtube.com//watch?v=KxGPe1jD-qY', 'rec_creator': 'National Gallery of Canada'}
{'seed_title': 'Political Correctness and Vacuous Wokeness: Douglas Murray debates Sylvana Simons', 'seed_url': 'https://www.youtube.com//watch?v=m2zZMg7SNWA', 'seed_creator': 'Sky News Australia', 'rec_title': 'Comedian Andrew Doyle takes apart woke comedy and political correctness - BQ #5', 'rec_url': 'https://www.youtube.com//watch?v=IqQBLIzDDUQ', 'rec_creator': 'The Sun'}
{'seed_title': 'Political Correctness and Vacuous Wokeness: Douglas Murray debates Sylvana Simons', 'seed_url': 'https://www.youtube.com//watch?v=m2zZMg7SNWA', 'seed_creator': 'Sky News Australia', 'rec_title': 'Comedian Andrew Doyle takes apart woke com

In [139]:
df[:30]

,seed_title,seed_url,seed_creator,rec_title,rec_url,rec_creator
100,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson: “There was plenty of motivati...,https://www.youtube.com//watch?v=yZYQpge1W5s,British GQ
101,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson Predicts The Death Of Mainstre...,https://www.youtube.com//watch?v=1Gp7xNnW5n8,Chris Williamson
102,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,"Jordan Peterson debate on the gender pay gap, ...",https://www.youtube.com//watch?v=aMcjxSThD54,Channel 4 News
103,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the #Metoo Moment,https://www.youtube.com//watch?v=g8GSlP2yCD8,National Post
104,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Joe Rogan Experience #1408 - Ed Calderon,https://www.youtube.com//watch?v=xPBejhoKlb8,PowerfulJRE
0,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the 'backlash against mascu...,https://www.youtube.com//watch?v=ShS4uEY2Jw8,BBC News
1,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,"Jordan Peterson debate on the gender pay gap, ...",https://www.youtube.com//watch?v=aMcjxSThD54,Channel 4 News
2,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson discusses whether men and wome...,https://www.youtube.com//watch?v=Iy4vq8RdPGU,Jeremy Vine on 5 - Official Channel
3,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the #Metoo Moment,https://www.youtube.com//watch?v=g8GSlP2yCD8,National Post
4,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson discusses whether men and wome...,https://www.youtube.com//watch?v=Iy4vq8RdPGU,Jeremy Vine on 5 - Official Channel


In [127]:
df

,seed_title,seed_url,seed_creator,rec_title,rec_url,rec_creator
100,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson: “There was plenty of motivati...,https://www.youtube.com//watch?v=yZYQpge1W5s,British GQ
101,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson Predicts The Death Of Mainstre...,https://www.youtube.com//watch?v=1Gp7xNnW5n8,Chris Williamson
102,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,"Jordan Peterson debate on the gender pay gap, ...",https://www.youtube.com//watch?v=aMcjxSThD54,Channel 4 News
103,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the #Metoo Moment,https://www.youtube.com//watch?v=g8GSlP2yCD8,National Post
104,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Joe Rogan Experience #1408 - Ed Calderon,https://www.youtube.com//watch?v=xPBejhoKlb8,PowerfulJRE
0,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the 'backlash against mascu...,https://www.youtube.com//watch?v=ShS4uEY2Jw8,BBC News
1,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,"Jordan Peterson debate on the gender pay gap, ...",https://www.youtube.com//watch?v=aMcjxSThD54,Channel 4 News
2,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson discusses whether men and wome...,https://www.youtube.com//watch?v=Iy4vq8RdPGU,Jeremy Vine on 5 - Official Channel
3,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson on the #Metoo Moment,https://www.youtube.com//watch?v=g8GSlP2yCD8,National Post
4,Jordan Peterson on Women's Studies (from Joe R...,https://www.youtube.com/watch?v=88KJ5rgCNmk,PowerfulJRE,Jordan Peterson discusses whether men and wome...,https://www.youtube.com//watch?v=Iy4vq8RdPGU,Jeremy Vine on 5 - Official Channel


In [140]:
df.to_csv("run2.csv")

In [130]:
counter = 55

In [131]:
counter

55